# Importing & Installing libraries

In [ ]:
!pip install refinitiv-data

In [ ]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
from refinitiv.data.content import symbol_conversion
from refinitiv.data.discovery import Peers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from scipy import stats

# Stablishing the LSEG session

In [ ]:
rd.open_session(name='platform.rdp',
                config_name="./refinitiv-data.config.json")

# Analist recomendations

In [ ]:
target_comp = ['APA.OQ']

recommendations_analyst = rd.get_data(target_comp, 'TR.RecLabel();TR.NumOfRecommendations();TR.NumEstRevisingUp;TR.NumEstRevisingDown', {
                  'WP': '30d', 'EstimateMeasure': 'EPS'})

recommendations_analyst

In [ ]:
analyst_info = rd.get_data(target_comp, ['TR.EPSEstValue.date',
                                         'TR.EPSEstValue',
                                         'TR.EPSEstValue.brokername',
                                         'TR.EPSEstValue.analystname'])
analyst_info

# Getting Peers & comparables valuation

In [ ]:
target_company = ['APA.OQ']
comparables = ['FANG.OQ','DVN.N','MRO.N','COP.N','EOG.N']

fields_target = ['TR.TickerSymbol','TR.CommonName','TR.PriceClose','TR.EBITDA','TR.PE','TR.EVToEBITDA','TR.RevenueActValue',
                  'TR.EVToSales','TR.SharesOutstanding','TR.EPSActValue','TR.CompanyMarketCap',
                  'TR.CashandSTInvestments','TR.F.DebtTot']

data_target = rd.get_data(fields = fields_target,
          universe=target_company)

In [ ]:
data_target.columns

In [ ]:
def get_comparable_valuation(target_company,universe_comparables,drop_na = False,verbose = False):

  fields_target = ['TR.TickerSymbol','TR.CommonName','TR.PriceClose','TR.EBITDA','TR.PE','TR.EVToEBITDA','TR.RevenueActValue',
                   'TR.EVToSales','TR.SharesOutstanding','TR.EPSActValue','TR.CompanyMarketCap',
                   'TR.CashandSTInvestments','TR.F.DebtTot']
  fields_comparables = ['TR.TickerSymbol','TR.CommonName','TR.PE','TR.EVToEBITDA','TR.EVToSales']

  data_target = rd.get_data(fields = fields_target,
            universe=target_company)

  data_comparables = rd.get_data(fields = fields_comparables,
            universe=universe_comparables)

  def correct_negative_ratios(x):
    if pd.isna(x) or x<0:
      return np.NaN
    else:
      return x

  data_comparables['Enterprise Value To Sales (Daily Time Series Ratio)'] = data_comparables['Enterprise Value To Sales (Daily Time Series Ratio)'].apply(correct_negative_ratios)
  data_comparables['Enterprise Value To EBITDA (Daily Time Series Ratio)'] = data_comparables['Enterprise Value To EBITDA (Daily Time Series Ratio)'].apply(correct_negative_ratios)
  data_comparables['P/E (Daily Time Series Ratio)'] = data_comparables['P/E (Daily Time Series Ratio)'].apply(correct_negative_ratios)

  revenue_target = data_target['Revenue - Actual'].values[0]
  cash_target = data_target['Cash and Short Term Investments'].values[0]
  debt_target = data_target['Debt - Total'].values[0]
  shares_target = data_target['Outstanding Shares'].values[0]
  ebitda_target = data_target['EBITDA'].values[0]
  earnings_target = data_target['Earnings Per Share - Actual'].values[0]*shares_target
  price_target =  data_target['Price Close'].values[0]
  ticker_target = data_target['Ticker Symbol'].values[0]
  name_target = data_target['Company Common Name'].values[0]

  EV_REV_comp = data_comparables['Enterprise Value To Sales (Daily Time Series Ratio)']

  implied_price_1 = (revenue_target*EV_REV_comp + cash_target - debt_target)/shares_target

  EV_EBITDA_comp = data_comparables['Enterprise Value To EBITDA (Daily Time Series Ratio)']

  implied_price_2 = (ebitda_target*EV_EBITDA_comp + cash_target - debt_target)/shares_target

  implied_price_3 = (data_comparables['P/E (Daily Time Series Ratio)']*earnings_target)/shares_target


  output = pd.DataFrame()
  output['Comparables'] = universe_comparables
  output['Ticker'] = data_comparables['Ticker Symbol']
  output['Name'] = data_comparables['Company Common Name']
  output['Price_EV/Sales'] = implied_price_1
  output['Price_EV/EBITDA'] = implied_price_2
  output['Price_PE'] = implied_price_3
  output['target_close_price'] = price_target

  output['Price_EV/Sales'] = output['Price_EV/Sales'].apply(correct_negative_ratios)
  output['Price_EV/EBITDA'] = output['Price_EV/EBITDA'].apply(correct_negative_ratios)
  output['Price_PE'] = output['Price_PE'].apply(correct_negative_ratios)

  total_mean = np.mean(output[['Price_EV/Sales','Price_EV/EBITDA','Price_PE','target_close_price']].mean())

  columns = output.columns

  mean_prices = ['MEAN']*3 + output[['Price_EV/Sales','Price_EV/EBITDA','Price_PE','target_close_price']].mean().values.tolist()
  output.loc[len(output)] = mean_prices
  median_prices = ['MEDIAN']*3 + output[['Price_EV/Sales','Price_EV/EBITDA','Price_PE','target_close_price']].median().values.tolist()
  output.loc[len(output)] = median_prices

  if verbose:
    print(f'- The mean implied price of {name_target} is equal to {total_mean:0.2f}')
    print(f'- The current closing price of {name_target} is {price_target:0.2f}')

    if price_target < total_mean:
      print(f'- RECOMMENDATION: {name_target} is UNDERVALUED suggestion to BUY')
    else:
      print(f'- RECOMMENDATION: {name_target} is OVERVALUED suggestion to SELL')

  return output

In [ ]:
target_comp = ['APA.OQ']
comparables = ['FANG.OQ','DVN.N','MRO.N','COP.N','EOG.N']

In [ ]:
comparables = get_comparable_valuation(target_comp,comparables)

In [ ]:
comparables

# Use the refinitiv Peer functionality

In [ ]:
from pprint import pprint

In [ ]:
_peers = Peers('APA.OQ')
_peers = [*_peers]

In [ ]:
pprint(_peers,width=50,compact=True)

In [ ]:
comparables_2 = get_comparable_valuation(target_comp,_peers)

In [ ]:
comparables_2.tail(10)